In [1]:
import sys
import pandas as pd
from pathlib import Path
import logging
from datetime import date

# Simulate __file__ in Jupyter
__file__ = str(Path.cwd() / "tiingo_maintainance_test.ipynb")

# Get the path to your secrets directory
project_root = Path(__file__).parents[1]
sys.path.insert(0, str(project_root))

In [2]:
# must come after path is set
from util.to_postgres import PgHook
from util.tiingo_manager import TiingoDataManager

In [3]:
print(sys.path[0])

/home/rthomas/thomasr/quant_lab


In [4]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('tiingo_load.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [5]:
# instantiate Postgres database utility
db = PgHook()

In [6]:
# Instantiate Tiingo class
tdm = TiingoDataManager()

2025-12-02 13:47:57,753 - INFO - Initialize TiingoDataManager with rate limiting


#### Update Symbols Table

In [7]:
# Get all tickers from Tiingo
print("Fetching ticker universe...")
tiingo_df = tdm.get_all_tickers(include_delisted=False, filter_junk=True)

2025-12-02 13:48:01,949 - INFO - Fetching all tickers from Tiingo...


Fetching ticker universe...


2025-12-02 13:48:02,437 - INFO - Filtered out 4746 junk tickers (warrants, units, preferreds)


In [8]:
# Get current tickers and active status
cur_symbols_df = db.psy_query("select ticker, is_active as is_active_old from symbols")

# Merge in current active flag to new records
existing_df = tiingo_df.merge(
    cur_symbols_df[['ticker', 'is_active_old']],
    on='ticker',
    how='inner'
)

In [9]:
# DEBUG
cur_symbols_df.head()

,ticker,is_active_old
0,A,1
1,AA,1
2,AAA,1
3,GSPC,1
4,DJI,1


In [10]:
# DEBUG
len(cur_symbols_df)

12999

In [ ]:
# DEBUG: temporarily stash original symbols tickers
db.alc_df_2_db_r(cur_symbols_df, 'temp_orig_symbols')

In [ ]:
# DEBUG: save tiingo download
db.alc_df_2_db_r(tiingo_df, 'temp_tiingo')

In [11]:
# DEBUG
existing_df.head()

,ticker,company_name,exchange,asset_type,start_date,end_date,is_active,is_etf,is_active_old
0,A,None,NYSE,stock,1999-11-18,None,1,0,1
1,AA,None,NYSE,stock,2016-10-18,None,1,0,1
2,AAA,None,NYSE ARCA,etf,2020-09-09,None,1,1,1
3,AAAU,None,BATS,etf,2018-08-15,None,1,1,1
4,AAC,None,NYSE,stock,2021-03-25,None,1,0,0


In [9]:
# Update newly inactive tickers and put in end date  
newly_inactive_mask = (existing_df['is_active'] == 0) & (existing_df['is_active_old'] == 1)
existing_df.loc[newly_inactive_mask, 'end_date'] = date.today()

In [10]:
# DEBUG: Newly inactive tickers
newly_inactive_mask.sum()

0

In [22]:
# DEBUG
newly_inactive_mask.head()

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [23]:
# DEBUG
len(existing_df)

12967

In [ ]:
# DEBUG
existing_df.head()

,ticker,company_name,exchange,asset_type,start_date,end_date,is_active,is_etf,is_active_old
0,A,None,NYSE,stock,1999-11-18,None,1,0,1
1,AA,None,NYSE,stock,2016-10-18,None,1,0,1
2,AAA,None,NYSE ARCA,etf,2020-09-09,None,1,1,1
3,AAAU,None,BATS,etf,2018-08-15,None,1,1,1
4,AAC,None,NYSE,stock,2014-10-02,None,1,0,1


In [11]:
# Get dataframe of newly inactive tickers
updates_df = existing_df.loc[
    existing_df['end_date'].notna(),
    ['ticker', 'is_active', 'end_date']
]
print(f"Newly inactive tickers to update: {len(updates_df)}")

Newly inactive tickers to update: 0


In [12]:
if len(updates_df) > 0:
    # update symbols table in database
    for _, row in updates_df.iterrows():
        sql = """
            UPDATE symbols 
            SET is_active = %s, end_date = %s
            WHERE ticker = %s
        """
        db.execute_sql(sql, (row['is_active'], row['end_date'], row['ticker']))

    logger.info(f"Updated {len(updates_df)} symbols with inactive status")

else:
    logger.info("No updates on 'is_active' to symbols.")

2025-12-02 13:48:35,817 - INFO - No updates on 'is_active' to symbols.


In [13]:
# Find new tickers (exclude known junk)
excluded_df = db.psy_query("select ticker from excluded_tickers")
new_tickers_df = tiingo_df[
    ~tiingo_df['ticker'].isin(cur_symbols_df['ticker']) &
    ~tiingo_df['ticker'].isin(excluded_df['ticker'])
]

print(f"New tickers to add: {len(new_tickers_df)}")

New tickers to add: 7


In [14]:
# Check all_symbols table FIRST (fast - no API calls)
all_symbols_df = db.psy_query('select ticker, company_name from all_symbols')
new_tickers_df = tdm.merge_names(new_tickers_df, all_symbols_df)

2025-12-02 13:48:47,420 - INFO - Enriched 1 company names from all_symbols table
2025-12-02 13:48:47,420 - INFO - Remaining nulls: 6


In [15]:
# Only enrich tickers that STILL have no name
needs_enrichment = new_tickers_df[new_tickers_df['company_name'].isna()]['ticker'].tolist()
print(f"Tickers needing API enrichment: {len(needs_enrichment)}")

Tickers needing API enrichment: 6


In [16]:
# Get company names for new tickers
enriched_ticker_df = tdm.enrich_company_names(new_tickers_df['ticker'].to_list())

2025-12-02 13:49:06,866 - INFO - Null company names: 0, Empty company names: 0


Completed 7 iterations, 0 errors


In [17]:
# DEBUG
print(enriched_ticker_df)

  ticker                                       company_name
0   DDFD  Innovator Equity Dual Directional 15 Buffer ET...
1   DDTD  Innovator Equity Dual Directional 10 Buffer ET...
2   FMTO                             Femto Technologies Inc
3   GRDX                            Entero Therapeutics Inc
4   PTPI                         Petros Pharmaceuticals Inc
5    SMQ          Tradr 1X Short Innovation 100 Monthly ETF
6   XAGE                      Longevity Health Holdings Inc


In [18]:
# Merge company names into tiingo_df to make new_symbols_df
new_tickers_df = tdm.merge_names(new_tickers_df, enriched_ticker_df)

2025-12-02 13:49:30,228 - INFO - Enriched 6 company names from all_symbols table
2025-12-02 13:49:30,229 - INFO - Remaining nulls: 0


In [19]:
# After getting new_tickers_df, before filtering
# Fix mislabeled ETFs - check company name only
etf_mask = (
    (new_tickers_df['asset_type'] == 'stock') &
    new_tickers_df['company_name'].notna() &
    (
        new_tickers_df['company_name'].str.contains(' ETF ', case=False, na=False) |
        new_tickers_df['company_name'].str.lower().str.endswith('etf')
    )
)

new_tickers_df.loc[etf_mask, 'asset_type'] = 'etf'
new_tickers_df.loc[etf_mask, 'is_etf'] = 1

logger.info(f"Corrected {etf_mask.sum()} ETFs mislabeled as stocks")

2025-12-02 13:49:35,253 - INFO - Corrected 3 ETFs mislabeled as stocks


In [20]:
# DEBUG
new_tickers_df.head(30)

,ticker,company_name,exchange,asset_type,start_date,end_date,is_active,is_etf
6546,DDFD,Innovator Equity Dual Directional 15 Buffer ET...,BATS,etf,2025-12-01,None,1,1
6574,DDTD,Innovator Equity Dual Directional 10 Buffer ET...,BATS,etf,2025-12-01,None,1,1
9493,FMTO,Femto Technologies Inc,NASDAQ,stock,2022-05-31,None,1,0
11016,GRDX,Entero Therapeutics Inc,NASDAQ,stock,2016-10-11,None,1,0
20471,PTPI,"Petros Pharmaceuticals, Inc.",NASDAQ,stock,2020-12-02,None,1,0
23119,SMQ,Tradr 1X Short Innovation 100 Monthly ETF,BATS,etf,2025-12-01,None,1,1
27515,XAGE,Longevity Health Holdings Inc,NASDAQ,stock,2021-09-23,None,1,0


In [ ]:
# DEBUG: Get the actual full value
full_name = new_tickers_df[new_tickers_df['ticker'] == 'DLNV']['company_name'].iloc[0]
print(full_name)
print(f"Length: {len(full_name)}")

FT Vest U.S. Equity Dual Directional Buffer ETF November
Length: 56


In [21]:
# After all enrichment attempts...
new_symbols_df = new_tickers_df[
    new_tickers_df['company_name'].notna() & 
    (new_tickers_df['company_name'] != '') &
    (new_tickers_df['company_name'].str.lower() != 'null')
]

In [22]:
# Save junk tickers so we skip them next time
junk_tickers = new_tickers_df[
    new_tickers_df['company_name'].isna() | 
    (new_tickers_df['company_name'] == '') |
    (new_tickers_df['company_name'].str.lower() == 'null')
]['ticker'].tolist()

if junk_tickers:
    for ticker in junk_tickers:
        sql = """
            INSERT INTO excluded_tickers (ticker, reason)
            VALUES (%s, %s)
            ON CONFLICT (ticker) DO NOTHING
        """
        db.execute_sql(sql, (ticker, 'no_company_name'))
    logger.info(f"Added {len(junk_tickers)} to excluded_tickers")

In [23]:
new_symbols_df.head(30)

,ticker,company_name,exchange,asset_type,start_date,end_date,is_active,is_etf
6546,DDFD,Innovator Equity Dual Directional 15 Buffer ET...,BATS,etf,2025-12-01,None,1,1
6574,DDTD,Innovator Equity Dual Directional 10 Buffer ET...,BATS,etf,2025-12-01,None,1,1
9493,FMTO,Femto Technologies Inc,NASDAQ,stock,2022-05-31,None,1,0
11016,GRDX,Entero Therapeutics Inc,NASDAQ,stock,2016-10-11,None,1,0
20471,PTPI,"Petros Pharmaceuticals, Inc.",NASDAQ,stock,2020-12-02,None,1,0
23119,SMQ,Tradr 1X Short Innovation 100 Monthly ETF,BATS,etf,2025-12-01,None,1,1
27515,XAGE,Longevity Health Holdings Inc,NASDAQ,stock,2021-09-23,None,1,0


In [24]:
# Insert new symbol records to table - this needs to be changed to only upsert NEW tickers
print("Saving symbols...")
failed_symbols = tdm.upsert_symbols(new_symbols_df)
logger.info(f"Failed symbols inserted: {len(failed_symbols)}")

2025-12-02 13:50:24,055 - INFO - Upserting 7 symbols...
2025-12-02 13:50:24,136 - INFO - Processed 7/7 symbols
2025-12-02 13:50:24,137 - INFO - Successfully upserted 7 symbols, 0 failed
2025-12-02 13:50:24,137 - INFO - Failed symbols inserted: 0


Saving symbols...


#### Update Stocks Table

In [25]:
# Get current stock records
cur_stock_df = db.psy_query("select * from stocks")

# Get stocks out of refreshed symbols table
new_stock_df = db.psy_query("select ticker, company_name, exchange from symbols where asset_type = 'stock'")

In [26]:
# Find new stock records that are not in current stock records
addl_stocks_df = new_stock_df[~new_stock_df['ticker'].isin(cur_stock_df['ticker'])]

In [27]:
# DEBUG
len(addl_stocks_df)

1

In [28]:
if len(addl_stocks_df) == 0:
    logger.info("No new stocks to add")
else:
    # Get industry & sector from yFinance for new stock records
    yf_enriched_df = tdm.yfinance_metadata(addl_stocks_df['ticker'].tolist())

    # DEBUG: See what's returned from yFinance
    print(f"yFinance returned {len(yf_enriched_df)} rows")
    print(f"yFinance columns: {yf_enriched_df.columns.tolist()}")

    if len(yf_enriched_df) > 0 and 'industry' in yf_enriched_df.columns:
        # merge yFinance data into addl_stocks_df
        addl_stocks_df = addl_stocks_df.merge(
            yf_enriched_df[['ticker', 'industry', 'sector']],
            on='ticker',
            how='left'
        )
    else:
        logger.warning("yFinance returns no usable data")
        addl_stocks_df['industry'] = None
        addl_stocks_df['sector'] = None

    # Get industry & sector from FMP for any remaining null or empty records
    missing_mask = addl_stocks_df['industry'].isna() | (addl_stocks_df['industry'] == '')
    tickers_needing_fmp = addl_stocks_df.loc[missing_mask, 'ticker'].tolist()

    if tickers_needing_fmp:
        logger.info(f"Fetching FMP data for {len(tickers_needing_fmp)} tickers missing industry & sector")
        fmp_enriched_df = tdm.fetch_industry_sector(tickers_needing_fmp)

        # DEBUG: See what's returned from FMP
        print(f"FMP returned {len(fmp_enriched_df)} rows")
        print(f"FMP columns: {fmp_enriched_df.columns.tolist()}")

        # Only merge if we got data back
        if len(fmp_enriched_df) > 0:
            addl_stocks_df = addl_stocks_df.merge(
                fmp_enriched_df[['ticker', 'industry', 'sector']],
                on='ticker',
                how='left',
                suffixes=('', '_fmp')
            )

            addl_stocks_df['industry'] = addl_stocks_df['industry'].fillna(addl_stocks_df['industry_fmp'])
            addl_stocks_df['sector'] = addl_stocks_df['sector'].fillna(addl_stocks_df['sector_fmp'])

            addl_stocks_df = addl_stocks_df.drop(columns=['industry_fmp', 'sector_fmp'], errors='ignore')

        else:
            logger.warning("FMP returned no data for any tickers")

    # Set field order
    stocks_df = addl_stocks_df[['ticker', 'company_name', 'industry', 'sector', 'exchange']]

    # Save junk stocks missing industry data so we skip them next time
    junk_stocks = stocks_df[
        stocks_df['industry'].isna() | 
        (stocks_df['industry'] == '')
    ]['ticker'].tolist()

    if junk_stocks:
        for ticker in junk_stocks:
            sql = """
                INSERT INTO excluded_tickers (ticker, reason)
                VALUES (%s, %s)
                ON CONFLICT (ticker) DO NOTHING
            """
            db.execute_sql(sql, (ticker, 'no_industry_data'))

        # Delete from symbols table as well
        delete_sql = """
            DELETE FROM symbols
            WHERE ticker in (SELECT ticker from excluded_tickers)
        """
        db.execute_sql(delete_sql)
        logger.info(f"Added {len(junk_stocks)} to excluded_tickers and removed from symbols")

    # After all enrichment attempts...
    clean_stocks_df = stocks_df[
        stocks_df['industry'].notna() & 
        (stocks_df['industry'] != '')
    ]

    # Save new stocks data to table
    failed_stocks = tdm.upsert_stocks(clean_stocks_df)

2025-12-02 13:51:05,880 - INFO - Fetching metadata from yFinance...
2025-12-02 13:51:06,280 - INFO - Found 1 tickers from yFinance.
2025-12-02 13:51:06,284 - INFO - Upserting 1 stocks...
2025-12-02 13:51:06,305 - INFO - Processed 1/1 symbols
2025-12-02 13:51:06,306 - INFO - Successfully upserted 1 symbols, 0 failed


yFinance returned 1 rows
yFinance columns: ['ticker', 'company_name_yf', 'is_etf_yf', 'industry', 'sector', 'exchange_yf']


#### Get Price Data

In [29]:
# Get max trade date in ohlcv table
max_trade_date = db.psy_query("select (max(trade_date)+1) as start_date from ohlcv")['start_date'].iloc[0]
print(max_trade_date)

2025-11-29


In [30]:
# Get current symbols - only select active tickers.  Additional fields for potential diagnosis.
# indices are omitted because they don't exist within Tiingo
symbols_df = db.psy_query("select ticker, company_name, asset_type from symbols where is_active = 1 and asset_type != 'index'")

In [31]:
# Get pricing data - This is a 200 minute operation for a day's prices
print("Downloading price data...")
ticker_list = symbols_df['ticker'].tolist() 
failed_prices = tdm.download_price_data(ticker_list, max_trade_date)
# failed_prices = tdm.download_price_data(ticker_list, '2025-11-18', '2025-11-21')  # was used to backfill a hole in the data

2025-12-02 13:51:42,428 - INFO - Starting price data download for 10166
2025-12-02 13:51:42,459 - INFO - Processing batch 1.0/102


2025-12-02 13:53:11,328 - ERROR - Failed to download FMTO: "None of ['date'] are in the columns"
2025-12-02 13:53:41,371 - INFO - Inserted 99 rows to staging
2025-12-02 13:53:41,372 - INFO - Processing batch 2.0/102
2025-12-02 13:55:41,387 - INFO - Inserted 100 rows to staging
2025-12-02 13:55:41,388 - INFO - Processing batch 3.0/102
2025-12-02 13:57:41,385 - INFO - Inserted 100 rows to staging
2025-12-02 13:57:41,386 - INFO - Processing batch 4.0/102
2025-12-02 13:58:54,583 - ERROR - Failed to download LSB: "None of ['date'] are in the columns"
2025-12-02 13:59:23,347 - ERROR - Failed to download AMBC: "None of ['date'] are in the columns"
2025-12-02 13:59:41,420 - INFO - Inserted 98 rows to staging
2025-12-02 13:59:41,421 - INFO - Processing batch 5.0/102
2025-12-02 13:59:42,587 - ERROR - Failed to download NFINU: "None of ['date'] are in the columns"
2025-12-02 14:00:22,222 - ERROR - Failed to download NFINW: "None of ['date'] are in the columns"
2025-12-02 14:01:41,441 - INFO - Ins

In [32]:
# Validate and move price data to ohlcv table
print("Validating and moving to production...")
tdm.validate_and_move_staging()

2025-12-02 17:28:39,328 - INFO - Validating staging data
2025-12-02 17:28:39,372 - INFO - Staging table contains 10,130 rows
2025-12-02 17:28:39,372 - INFO - Validation passed - no issues found
2025-12-02 17:28:39,373 - INFO - Moving staging data to production...
2025-12-02 17:28:39,449 - INFO - Moved 10,130 rows to production
2025-12-02 17:28:39,468 - INFO - Successfully moved staging to production


Validating and moving to production...


True

In [33]:
# D --- Mark stale tickers as inactive
print("Checking for stale tickers...")

# First count how many will be affected
count_query = """
    SELECT COUNT(*) as cnt
    FROM symbols s
    WHERE s.is_active = 1
      AND s.asset_type != 'index'
      AND s.date_loaded < CURRENT_DATE - INTERVAL '30 days'
      AND s.ticker NOT IN (
          SELECT DISTINCT ticker 
          FROM ohlcv 
          WHERE trade_date >= CURRENT_DATE - INTERVAL '30 days'
      )
"""
stale_count = db.psy_query(count_query)['cnt'].iloc[0]
logger.info(f"Found {stale_count} stale tickers (no prices in 30 days)")

if stale_count > 0:
    # Now run the update
    update_sql = """
        UPDATE symbols s
        SET 
            is_active = 0,
            end_date = COALESCE(
                (SELECT MAX(trade_date) FROM ohlcv WHERE ticker = s.ticker),
                CURRENT_DATE
            )
        WHERE s.is_active = 1
          AND s.asset_type != 'index'
          AND s.date_loaded < CURRENT_DATE - INTERVAL '30 days'
          AND s.ticker NOT IN (
              SELECT DISTINCT ticker 
              FROM ohlcv 
              WHERE trade_date >= CURRENT_DATE - INTERVAL '30 days'
          )
    """
    db.execute_sql(update_sql)
    logger.info(f"Marked {stale_count} tickers as inactive")
else:
    logger.info("No stale tickers to mark inactive")

Checking for stale tickers...


2025-12-02 17:28:44,472 - INFO - Found 0 stale tickers (no prices in 30 days)
2025-12-02 17:28:44,472 - INFO - No stale tickers to mark inactive


#### Patch Up Not Associated With Process

In [7]:
# Get the mislabeled ETFs added on 11/21
new_etfs = db.psy_query("""
    SELECT ticker FROM symbols 
    WHERE date_loaded = '2025-11-21' 
    AND asset_type = 'etf'
""")

print(f"Backfilling prices for {len(new_etfs)} ETFs...")

Backfilling prices for 118 ETFs...


In [8]:
# Fetch 10-year history
failed = tdm.download_price_data(new_etfs['ticker'].tolist(), start_date='2015-01-01')

2025-11-23 17:39:31,398 - INFO - Starting price data download for 118
2025-11-23 17:39:31,429 - INFO - Processing batch 1.0/2
2025-11-23 17:40:23,098 - ERROR - Failed to download MELT: "None of ['date'] are in the columns"
2025-11-23 17:40:41,102 - ERROR - Failed to download BA-P: "None of ['date'] are in the columns"
2025-11-23 17:40:54,302 - ERROR - Failed to download PA-P: "None of ['date'] are in the columns"
2025-11-23 17:41:31,154 - INFO - Inserted 50593 rows to staging
2025-11-23 17:41:31,154 - INFO - Processing batch 2.0/2
2025-11-23 17:41:37,516 - ERROR - Failed to download UA-P: "None of ['date'] are in the columns"
2025-11-23 17:41:52,063 - INFO - Inserted 7784 rows to staging
2025-11-23 17:41:52,064 - INFO - Download complete.  Failed tickers: 4


In [9]:
# Validate and move to production
tdm.validate_and_move_staging()

2025-11-23 17:41:58,035 - INFO - Validating staging data
2025-11-23 17:41:58,123 - INFO - Staging table contains 58,377 rows
2025-11-23 17:41:58,124 - INFO - Validation passed - no issues found
2025-11-23 17:41:58,125 - INFO - Moving staging data to production...
2025-11-23 17:42:00,897 - INFO - Moved 58,377 rows to production
2025-11-23 17:42:00,916 - INFO - Successfully moved staging to production


True

In [10]:
# Get the missing indices added on 11/23
index_df = db.psy_query("""
    SELECT ticker FROM symbols 
    WHERE date_loaded = '2025-11-23' 
    AND asset_type = 'index'
""")

print(f"Backfilling prices for {len(index_df)} ETFs...")

Backfilling prices for 8 ETFs...


In [ ]:
# Fetch 10-year history - Ruh row.  Proprietary but we have the ETF equivalents.  A table index_mapping was created
failed = tdm.download_price_data(index_df['ticker'].tolist(), start_date='2015-01-01')

2025-11-23 18:15:56,249 - INFO - Starting price data download for 8
2025-11-23 18:15:56,269 - INFO - Processing batch 1.0/1
2025-11-23 18:15:56,406 - ERROR - b'{"detail":"Error: Ticker \'GSPC\' not found"}'
2025-11-23 18:15:56,407 - ERROR - Failed to download GSPC: 404 Client Error: Not Found for url: https://api.tiingo.com/tiingo/daily/GSPC/prices?format=json&resampleFreq=daily&startDate=2015-01-01
2025-11-23 18:15:57,582 - ERROR - b'{"detail":"Error: Ticker \'DJI\' not found"}'
2025-11-23 18:15:57,582 - ERROR - Failed to download DJI: 404 Client Error: Not Found for url: https://api.tiingo.com/tiingo/daily/DJI/prices?format=json&resampleFreq=daily&startDate=2015-01-01
2025-11-23 18:15:58,764 - ERROR - b'{"detail":"Error: Ticker \'NDX\' not found"}'
2025-11-23 18:15:58,768 - ERROR - Failed to download NDX: 404 Client Error: Not Found for url: https://api.tiingo.com/tiingo/daily/NDX/prices?format=json&resampleFreq=daily&startDate=2015-01-01
2025-11-23 18:15:59,970 - ERROR - b'{"detail"